## Predicting Good Amazon Reviews

For this demo, we will be using the [Amazon Fine Food Reviews Data](https://www.kaggle.com/snap/amazon-fine-food-reviews).  The Amazon Fine Food Reviews dataset consists of 568,454 food reviews Amazon users left up to October 2012.

## Import Data

We will begin by importing our review data into our H2O cluster.  In this case, I will be starting up an H2O cluster on my local computer.

In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.7.0_80"; Java(TM) SE Runtime Environment (build 1.7.0_80-b15); Java HotSpot(TM) 64-Bit Server VM (build 24.80-b11, mixed mode)
  Starting server from /Users/megankurka/anaconda3/envs/py27/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/gv/w2f3zs_d33l3dt5j67k9nhhr0000gn/T/tmpmWHY4w
  JVM stdout: /var/folders/gv/w2f3zs_d33l3dt5j67k9nhhr0000gn/T/tmpmWHY4w/h2o_megankurka_started_from_python.out
  JVM stderr: /var/folders/gv/w2f3zs_d33l3dt5j67k9nhhr0000gn/T/tmpmWHY4w/h2o_megankurka_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_megankurka_7bn4og
H2O cluster total nodes:,1
H2O cluster free memory:,5.750 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
# Import Data
reviewsHex = h2o.import_file("./amazon-fine-food-reviews/Reviews.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
#' Add Target Column: "GoodReview"
reviewsHex["GoodReview"] = (reviewsHex["Score"] >= 4).ifelse("Yes", "No")

In [5]:
reviewsHex.head()

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,GoodReview
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1.30386e+09,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.,Yes
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1.34698e+09,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as """"Jumbo""",No
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres """"Natalia Corres",1,1,4,1.21902e+09,"""""Delight"""" says it a","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' """"The Lion, The Witch, and The Wardrobe"""" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witc",Yes
4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1.30792e+09,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.,No
5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham """"M. Wassir",0,0,5,1.35078e+09,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal.",Yes
6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1.34205e+09,Nice Taffy,"I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there was a bit too much red/black licorice-flavored pieces (just not my particular favorites). Between me, my kids, and my husband, this lasted only two weeks! I would recommend this brand of taffy -- it was a delightful treat.",Yes
7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1.34015e+09,Great! Just as good as the expensive brands!,"This saltwater taffy had great flavors and was very soft and chewy. Each candy was individually wrapped well. None of the candies were stuck together, which did happen in the expensive version, Fralinger's. Would highly recommend this candy! I served it at a beach-themed party and everyone loved it!",Yes
8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1.336e+09,"Wonderful, tasty taffy",This taffy is so good. It is very soft and chewy. The flavors are amazing. I would definitely recommend you buying it. Very satisfying!!,Yes
9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1.32201e+09,Yay Barley,Right now I'm mostly just sprouting this so my cats can eat the grass. They love it. I rotate it around with Wheatgrass and Rye too,Yes
10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1.35121e+09,Healthy Dog Food,This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.,Yes


In [6]:
reviewsHex["GoodReview"].table()

GoodReview,Count
No,124677
Yes,443777


In [7]:
reviewsHex.nrow

568454

In [8]:
# Filter so we can run this comfortably locally
# Comment this line out to train on the whole dataset

reviewsHex = reviewsHex[0:100000, :]

## Tokenize Words in Review

Our first step will be to tokenize the words in the review column. We will do this by creating a function called `tokenize`.  This will split the reviews into words and remove any stop words, small words, or words with numbers in them.

In [9]:
# Set Stop Words

STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

# We can also use packages like nltk for stop words

from nltk.corpus import stopwords
STOP_WORDS = set(stopwords.words('english'))

In [10]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [11]:
# Break job titles into sequence of words
words = tokenize(reviewsHex["Text"])

In [12]:
words.head()

C1
bought
several
vitality
canned
dog
food
products
found
good
quality


In [13]:
words.nrow

4218105

## Train Word2Vec Model

Now that we've tokenized our words, we can train a word2vec model.  We can use the `find_synonms` function to sanity check our word2vec model after training.

In [14]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [15]:
# Sanity check - find synonyms for the word 'coffee'
w2v_model.find_synonyms("coffee", count = 5)

OrderedDict([(u'coffees', 0.8039896488189697),
             (u'coffe', 0.7849918603897095),
             (u'starbucks', 0.7691516280174255),
             (u'gevalia', 0.7566434741020203),
             (u'espresso', 0.7507617473602295)])

In [16]:
# Sanity check - find synonyms for the word 'stale'
w2v_model.find_synonyms("stale", count = 5)

OrderedDict([(u'rancid', 0.7066834568977356),
             (u'moldy', 0.6781209707260132),
             (u'expired', 0.6481258869171143),
             (u'disappointing', 0.6162458658218384),
             (u'musty', 0.6065858602523804)])

Now that we have a word embedding for each word in our vocabulary, we will aggregate the words for each review using the `transform` function.  This will give us one aggregated word embedding for each review.

In [17]:
# Calculate a vector for each review
review_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [18]:
review_vecs

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.0927789,-0.0780789,0.0400514,0.078961,0.00621785,-0.183841,0.0509493,-0.0994213,-0.0343305,0.010621,0.00218504,0.0488486,-0.0995271,0.04161,-0.0239654,-0.211431,0.00839208,-0.0822046,-0.0172611,-0.122725,0.0173305,-0.0126777,0.0822573,0.0447836,0.0226046,-0.150754,-0.0202274,0.0826349,0.0411864,-0.00231425,0.0441491,-0.0662871,-0.0501635,0.0833037,0.00427557,0.053497,0.139145,0.0870243,-0.123576,0.0426633,-0.217072,0.0109024,0.110409,-0.00837098,0.0325347,0.0905762,-0.106751,0.0246276,0.103316,-0.00432012,-0.0849231,-0.131995,0.0656901,-0.105888,0.0551366,-0.0464983,-0.126418,-0.135266,-0.0152979,-0.0330269,-0.176351,-0.0782321,0.0109124,-0.0714267,0.129922,-0.0672304,-0.0109547,0.221899,0.179982,-0.0812537,-0.127889,0.119524,0.0570894,-0.0318549,0.0745752,-0.0429706,0.0922999,0.018533,0.102974,-0.0327891,-0.0141071,-0.0208345,0.125753,-0.0854937,-0.147191,-0.0114635,-0.0606758,-0.0526381,-0.0416607,0.0281737,-0.0202411,-0.219648,0.161414,0.262375,-0.0811713,0.0391989,0.0148403,0.00146639,-0.181178,-0.171293
-0.276033,0.0151484,0.0113293,0.0677438,-0.0905324,-0.118774,-0.0999573,0.125073,0.051549,0.0185647,-0.0383856,0.136076,0.192993,-0.062511,0.100237,-0.0218334,0.0439181,-0.237055,0.0889052,-0.018011,-0.0731727,-0.0683486,0.0459292,0.132132,-0.0988538,-0.0150929,-0.013194,0.150352,-0.0150005,-0.066652,-0.132323,-0.00393814,0.0117565,0.104176,0.0079745,0.0149131,0.196219,0.164746,-0.0922121,0.0097616,-0.0173784,-0.028756,0.254574,-0.283968,0.165734,0.0759202,-0.0449227,0.0166824,-0.0533121,0.171682,0.0518452,-0.0854528,0.0639833,0.0595394,0.0330438,0.0793302,-0.20656,-0.029051,-0.0258373,0.0854011,-0.176907,-0.0136716,-0.257297,0.0196326,0.0695244,0.054746,-0.00785023,0.166544,0.0979964,-0.0771203,-0.116782,0.108944,0.19234,0.118503,0.232888,-0.0798198,0.151042,0.0179298,-0.0448813,0.0679556,-0.167542,0.152458,0.0433542,0.0893348,-0.238303,0.00790076,-0.112416,-0.161058,-0.00302351,0.100816,-0.12185,-0.118988,0.0961822,0.118514,-0.0246701,0.0992028,0.0939886,0.0549001,-0.0921721,-0.140011
-0.134752,-0.0238701,0.0591484,-0.0536172,0.0340914,-0.043856,-0.0683675,0.0223286,0.0176382,0.034513,0.0265027,0.0285649,0.0419748,0.0340118,0.0171392,-0.00798347,-0.0237852,0.0323425,-0.0520917,0.00469608,-0.0351473,-0.0619472,0.054983,-0.020135,0.0118908,-0.0719161,0.00587609,0.218797,-0.0201062,-0.0775392,-0.117753,0.0643857,-0.0164111,0.0537017,-0.0119928,-0.0852235,0.113387,0.134459,-0.0989895,0.0487934,-0.116117,0.00537046,0.145004,-0.0995211,0.0166673,0.0260838,-0.0482889,0.144023,0.146793,0.0522942,-0.118236,-0.0705569,0.119445,0.0230592,0.00564553,0.00201827,-0.0131597,-0.250324,-0.0669661,-0.00745133,-0.0545438,-0.0887173,-0.0438246,-0.00846831,0.191976,0.0266586,-0.049748,0.113843,-0.13312,-0.0873035,-0.0578611,0.109486,0.0221456,0.0657341,-0.00800932,-0.0780109,0.106349,-0.0625127,0.00185641,-0.00238178,-0.0630112,0.117973,0.084102,0.0754979,-0.0258171,-0.075556,-0.179709,-0.151821,0.0253475,0.123583,-0.00576569,-0.279481,0.133486,0.22093,-0.045265,0.00444191,0.133329,0.020618,-0.105955,-0.0213722
-0.218111,-0.113166,0.0573562,-0.0972486,-0.0813074,0.0199388,0.175864,0.0571747,0.0559835,0.00238658,0.143382,0.0398854,0.0135824,0.0193048,0.0123202,-0.0178229,0.0122007,-0.00338829,-0.0508756,-0.0543691,0.0249526,0.0671062,0.190153,0.0867405,0.04638,-0.10859,0.104918,0.174844,-0.0453587,-0.0986282,0.0344955,-0.0529464,0.00632489,-0.0265126,-0.0121773,-0.0774013,0.124208,-0.0268536,-0.0847079,-0.116635,-0.149274,-0.0646061,0.175258,-0.11059,-0.0011508,0.164264,-0.0174991,-0.0644633,0.0850723,0.0201789,0.00886196,-0.0805991,0.198137,-0.0964649,0.

In [19]:
# Prepare training & validation data (keep only reviews made of known words)
valid_reviews = ~ review_vecs["C1"].isna()
data = reviewsHex[valid_reviews,:].cbind(review_vecs[valid_reviews,:])
data_split = data.split_frame(ratios=[0.8], seed = 1234)

## Train GBM Model to Predict Good Review

We will train a GBM model to predict whether the product has a positive or negative review using solely the aggregated word embeddings as predictors.

In [20]:
# Build a basic GBM model
from h2o.estimators import H2OGradientBoostingEstimator

gbm_model = H2OGradientBoostingEstimator()
gbm_model.train(x = review_vecs.names,
                y = "GoodReview", 
                training_frame = data_split[0], 
                validation_frame = data_split[1], 
                model_id = "gbm.hex")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [21]:
print("AUC: " + str(round(gbm_model.auc(valid = True), 3)))

AUC: 0.881


In [22]:
print("Accuracy: " + str(round(gbm_model.accuracy(valid = True)[0][1], 3)))

Accuracy: 0.85


In [23]:
def predict(reviews, w2v, gbm):
    words = tokenize(h2o.H2OFrame(reviews).ascharacter())
    reviews_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=reviews_vec))

In [24]:
# Predict!
print(predict(["Seriously this product was as tasteless as they come."], w2v_model, gbm_model))

print(predict(["Great taffy at a great price.  There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal."], w2v_model, gbm_model))

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,No,Yes
No,0.952698,0.0473021



None
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,No,Yes
Yes,0.0249092,0.975091



None


## Add Word Embeddings from Pretrained Glove Model

We will see if we can improve our performance by including word embeddings from a pretrained Glove model.  Glove is a different algorithm from Word2Vec but its goal is the same: convert words to word embeddings.  A pre-trained Glove model using Wikipedia articles can be found here http://nlp.stanford.edu/data/glove.6B.zip.

In [25]:
g6b = h2o.import_file("./glove.6B.100d.txt")

/Users/megankurka/anaconda3/envs/py27/lib/python2.7/site-packages/h2o/h2o.py:562: UserWarning: [Lwater.parser.ParseWriter$ParseErr;@7178676e
  warnings.warn(w)


Parse progress: |█████████████████████████████████████████████████████████| 100%


/Users/megankurka/anaconda3/envs/py27/lib/python2.7/site-packages/h2o/job.py:69: UserWarning: ParseError at file /Users/megankurka/Documents/Meetups/March_23_2017/glove/glove.6B.100d.txt  at byte offset 7674; error = 'Unmatched quote char "'
  warnings.warn(w)


In [26]:
glove_g6B = H2OWord2vecEstimator(pre_trained = g6b, vec_size = 100)
glove_g6B.train(training_frame = words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [27]:
# Calculate a vector for each review from the pretrained Glove model
pretrained_review_vecs = glove_g6B.transform(words, aggregate_method = "AVERAGE")

In [28]:
# Change Column Names to indicate from pretrained model
pretrained_review_vecs.names = ["pretrained_" + s for s in pretrained_review_vecs.names]

In [29]:
# Prepare training & validation data 
# Add the pretrained Glove word embeddings
data = data.cbind(pretrained_review_vecs[valid_reviews,:])
data_split = data.split_frame(ratios=[0.8], seed = 1234)

In [30]:
# Build a new GBM model with the added vectors

gbm_v2 = H2OGradientBoostingEstimator()
gbm_v2.train(x = review_vecs.names + pretrained_review_vecs.names,
             y="GoodReview", 
             training_frame = data_split[0], 
             validation_frame = data_split[1], 
             model_id = "gbm_v2.hex")

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [31]:
print("Original AUC: " + str(round(gbm_model.auc(valid = True), 3)))
print("New AUC: " + str(round(gbm_v2.auc(valid = True), 3)))

Original AUC: 0.881
New AUC: 0.88


## References

- Kaggle Amazon Reviews Data: https://www.kaggle.com/snap/amazon-fine-food-reviews

- J. McAuley and J. Leskovec. From amateurs to connoisseurs: modeling the evolution of user expertise through online reviews. WWW, 2013.

- Glove: https://nlp.stanford.edu/projects/glove/

- This script is based off of the [Craigslist Word2Vec Demo](https://github.com/h2oai/h2o-3/blob/master/h2o-py/demos/word2vec_craigslistjobtitles.ipynb)